# D&D Battle Sim
Basically running encounter simulations in python. Import two stat block, run battle scenarios, get the outcome. We're doing this without a graphical interface for the moment.
Running a basic rat vs commoner test here. Basically i'll just import two stat blocks and start with the basics of DND battle:  
- Roll for initiative
- whack whack whack
- end of fight

we'll output results as the battle unfolds for now.

In [64]:
# Imports
import requests
import json
import numpy as np
import math
import re

In [127]:
# Core functions

# returns a stat modifier
def getStatModifier(score):
    return math.floor(( score - 10 ) / 2)

# returns a random value in a list of options
def choose(options):
    return options[np.random.randint(0, len(options))]


# returns a random value between 1 and dtype
def rollDice(dType=20):
    return np.random.randint(1, dType+1)

# runs 1D6+4 format dice commands, outputs a dice roll format
def rollCommand(cmd):

    rolls = {
        'dice':[],
        'mods':[]
        }    

    diceRegex = r"(?i)(?P<dCount>\d+)d(?P<dType>\d+)(?P<dMod>[+-]\d+)?"
        
    #let's first evaluate the rolls 
    diceMatch = re.match(diceRegex, cmd)

    if diceMatch is not None:
        rolls['dice'] += [
            {
                "type":int(diceMatch.group("dType")),
                'roll':rollDice(int(diceMatch.group("dType")))
                }
            for d in range(int(diceMatch.group("dCount")))]

    if diceMatch.group("dMod") is not None:
        rolls['mods'].append(int(diceMatch.group("dMod")))

    return sum([d['roll'] for d in rolls['dice']]) + sum(rolls['mods'])
    

In [3]:
# importing stat blocks
baseURL = "https://www.dnd5eapi.co/api/monsters/"

commoner = json.loads(requests.get(baseURL + "commoner").text)
rat = json.loads(requests.get(baseURL + "rat").text)

In [4]:
def getInitiative(creature):
    return rollDice() + getStatModifier(creature["dexterity"])

getInitiative(rat)

9

In [5]:
# building the basics so they can fight
# we're putting them in a class that will help track relevant information later on 

class Battle(): 
    "Instances of Battle should take combat parameters as input, simulate a single combat and its outcome"

    creatures = {}
    roundCount = 0
    initiativeCount = 0
    winState = False

    def __init__(self, creatureList = []) -> None: 
        for creature in creatureList:  
            self.creatures[creature['index']] = {
                "initiativeCount": getInitiative(creature),
                "statBlock" : creature
            }

In [6]:
creatures = [{"initiativeCount" : 8,
    "team": "players",
    "statblock": commoner},
    {"initiativeCount" : 12, 
    "team": "monsters",
    "statblock": rat}
]

roundCount = 0
winState = False

def getOrderedCreatures(creatures):
    return sorted(creatures, key = lambda x: x['initiativeCount'], reverse=True)

while winState == False and roundCount < 10 :
        roundCount += 1
        print("starting round {}".format(roundCount))
        for creature in getOrderedCreatures(creatures):
            # call turn function here
            print('do stuff')



starting round 1
do stuff
do stuff
starting round 2
do stuff
do stuff
starting round 3
do stuff
do stuff
starting round 4
do stuff
do stuff
starting round 5
do stuff
do stuff
starting round 6
do stuff
do stuff
starting round 7
do stuff
do stuff
starting round 8
do stuff
do stuff
starting round 9
do stuff
do stuff
starting round 10
do stuff
do stuff


# Turn Option 1: Attacking
First action option is attacking. Basically pick an ennemy, make a roll against AC, calculate damage, and substract them to its total HP. 

In [138]:
creatures = [{"initiativeCount" : 8,
    "team": "players",
    "stats": commoner},
    {"initiativeCount" : 12, 
    "team": "monsters",
    "stats": rat},
    {"initiativeCount" : 12, 
    "team": "monsters",
    "stats": rat},
    {"initiativeCount" : 12, 
    "team": "monsters",
    "stats": rat}
]

# simulating the commoner's attack this will later be a part of the iterative process
attacker = creatures[0]
print("attacking with", attacker["stats"]["name"])

# legalTargets returns the index of creatures that can be legally attacked 
legalTargets = [index for index, creature in enumerate(creatures)
    if creature["team"] != attacker["team"]
    and creature['stats']['hit_points'] > 0 ]

# target is the index of creature from creatures that i'll be attacking
targetIndex = choose(legalTargets)

print("attacking", creatures[targetIndex]["stats"]["name"])

# I'll have to implement choosing between different actions at some point but for now let's just take the first
action = attacker["stats"]["actions"][0]
print("attack with", action["name"])

# making attack
if rollDice() + action["attack_bonus"] >= creatures[targetIndex]["stats"]["armor_class"]:
    print("attack hits!")

    # calculating damage
    totalDamage = 0
    for damageType in action["damage"]:
        totalDamage += rollCommand(damageType["damage_dice"])
        
    creatures[targetIndex]["stats"]["hit_points"] -= totalDamage
    
    print(creatures[targetIndex]["stats"]["hit_points"])
else: print('attack misses')




attacking with Commoner


ValueError: low >= high

In [136]:
action

{'name': 'Club',
 'desc': 'Melee Weapon Attack: +2 to hit, reach 5 ft., one target. Hit: 2 (1d4) bludgeoning damage.',
 'attack_bonus': 2,
 'damage': [{'damage_type': {'index': 'bludgeoning',
    'name': 'Bludgeoning',
    'url': '/api/damage-types/bludgeoning'},
   'damage_dice': '1d4'}]}